# 공시 정보 API 사용법

금융감독원의 https://dart.fss.or.kr/ 웹사이트에서는 국내 상장기업의 모든 공시 자료를 제공한다. 이 자료 목록은 RSS 를 통해서도 받을 수 있으나 여기에서 설명하듯이 API도 제공하고 있다.

## API 인증키 발급

DARTS API를 사용하려면 사용자 등록 후 API 인증키를 발급받아야 한다. 사용자 등록 및 인증키 발급은 다음 웹사이트를 참조한다.

* 사용자 등록: https://dart.fss.or.kr/dsag002/insertForm.do
* API 사용:  https://dart.fss.or.kr/dsap001/intro.do

인증키는 알파벳과 숫자로 이루어진 길이가 40인 문자열이다. 이 값은 API 호출시 `auth` 라는 http 파라미터로 제공해야 한다.


In [6]:
apikey = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # 실제 키값을 입력
type(apikey), len(apikey)

(str, 40)

## API 기능

DARTS API는 현재 다음과 같은 두 가지 기능을 제공한다.

* 기업 개황 제공
* 공시 정보 검색

현재는 공시 보고서 자체는 API로 제공하지 않으므로 공시 정보 검색 결과에서 제공하는 rcp_no 번호를 사용하여 개별적으로 크롤링하여야 한다.

## 기업 개황 

기업 개황 API 주소는 다음과 같다.

* http://dart.fss.or.kr/api/company.json?auth=xxx&crp_cd=xxx

여기에서 auth 값은 개인의 인증키 문자열이고 crp_cd 는 회사의 CRP 코드이다. CRP 코드는 DARTS 웹사이트의 "기업개황"에서 화사를 검색하면 결과 페이지에 회사별 공시 RSS 주소를 얻을 수 있는 링크를 제공한다.

상장사의 경우에는 6자리의 단축 ISIN 코드를 사용해도 된다.


https://dart.fss.or.kr/dsae001/main.do

<img src="https://datascienceschool.net/upfiles/cc5cd84fdbe04bf6b82f37ee6155907c.png" style="width:100%">

예를 들어 삼성전자의 CRP 코드는 `00126380` 이다.

In [27]:
url_company = "http://dart.fss.or.kr/api/company.json?auth={0}&crp_cd={1}"
crpcode = "00126380"
url = url_company.format(apikey, crpcode)

응답결과는 JSON 파일 형식으로 각 필드의 의미는 다음과 같다.

* `result`
 * 설명
* `err_code`
 * 	에러코드(오류 메시지 참조)
* `err_msg`
 * 	에러메시지(오류 메시지 참조)
* `crp_nm`
 * 	정식명칭
* `crp_nm_e`
 * 	영문명칭
* `crp_nm_i`
 * 	종목명(상장사) 또는 약식명칭(기타법인)
* `stock_cd`
 * 	상장회사인 경우 주식의 종목코드
* `ceo_nm`
 * 	대표자명
* `crp_cls`
 * 	법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
* `crp_no`
 * 	법인등록번호
* `bsn_no`
 * 	사업자등록번호
* `adr`
 * 	주소
* `hm_url`
 * 	홈페이지
* `ir_url`
 * 	IR홈페이지
* `phn_no`
 * 	전화번호
* `fax_no`
 * 	팩스번호
* `ind_cd`
 * 	업종코드
* `est_dt`
 * 	설립일(YYYYMMDD)
* `acc_mt`
 * 	결산월(MM)

In [28]:
import requests
import json
from konlpy.utils import pprint

response = requests.get(url)
data = json.loads(response.content)
pprint(data)

{acc_mt: 12,
 adr: 경기도 수원시 영통구  삼성로 129 (매탄동),
 bsn_no: 1248100998,
 ceo_nm: 권오현, 윤부근, 신종균,
 crp_cls: Y,
 crp_nm: 삼성전자(주),
 crp_nm_e: SAMSUNG ELECTRONICS CO,.LTD,
 crp_nm_i: 삼성전자,
 crp_no: 1301110006246,
 err_code: 000,
 err_msg: 정상,
 est_dt: 19690113,
 fax_no: 031-200-7538,
 hm_url: www.sec.co.kr,
 ind_cd: 264,
 ir_url: ,
 phn_no: 031-200-1114,
 stock_cd: 005930}


In [29]:
data["crp_no"]

u'1301110006246'

## 공시 정보 검색

공시 정보 검색 API 주소는 다음과 같다.

* http://dart.fss.or.kr/api/search.json?auth=xxx

여기에서 auth 값은 개인의 인증키 문자열이다. 이 뒤에 다음과 같은 파라미터를 추가하여야 한다.

* `crp_cd`	
 * 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
* `end_dt`	
 * 검색종료 접수일자(YYYYMMDD) : 없으면 당일
* `start_dt`	
 * 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
* `fin_rpt`	
 * 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
* `dsp_tp`	
 * 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J) 
* `bsn_tp`	
 * 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5) (상세 유형 참조)
* `sort`	
 * 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
* `series`	
 * 오름차순(asc), 내림차순(desc) 기본값 : desc
* `page_no`	
 * 페이지 번호(1~n) 기본값 : 1
* `page_set`	
 * 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
* `callback`	
 * 콜백함수명(JSONP용)

In [34]:
url_search = "http://dart.fss.or.kr/api/search.json?auth={0}&crp_cd={1}&start_dt=20000101"
crpcode = "00126380"
url = url_search.format(apikey, crpcode)

In [35]:
import requests
import json
from konlpy.utils import pprint

response = requests.get(url)
data = json.loads(response.content)
pprint(data)

{err_code: 000,
 err_msg: 정상,
 list: [{crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160707,
         rcp_no: 20160707800438,
         rmk: 유,
         rpt_nm: 기업설명회(IR)개최(안내공시)},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 지완구,
         rcp_dt: 20160707,
         rcp_no: 20160707000087,
         rmk: ,
         rpt_nm: 임원ㆍ주요주주특정증권등소유상황보고서},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160707,
         rcp_no: 20160707800065,
         rmk: 유,
         rpt_nm: 연결재무제표기준영업(잠정)실적(공정공시)},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160630,
         rcp_no: 20160630800581,
         rmk: 유,
         rpt_nm: 조회공시요구(풍문또는보도)에대한답변(부인)},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 유가증권시장본부,
         rcp_dt: 20160630,
         rcp

In [36]:
response = requests.get(url + "&page_no=2")
data = json.loads(response.content)
pprint(data)

{err_code: 000,
 err_msg: 정상,
 list: [{crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160609,
         rcp_no: 20160609800307,
         rmk: 유,
         rpt_nm: 중간(분기)배당을위한주주명부폐쇄(기준일)결정},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 조재문,
         rcp_dt: 20160609,
         rcp_no: 20160609000043,
         rmk: ,
         rpt_nm: 임원ㆍ주요주주특정증권등소유상황보고서},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160603,
         rcp_no: 20160603800057,
         rmk: 유,
         rpt_nm: 조회공시요구(풍문또는보도)에대한답변(부인)},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 유가증권시장본부,
         rcp_dt: 20160603,
         rcp_no: 20160603800019,
         rmk: 유,
         rpt_nm: 조회공시요구(풍문또는보도)},
        {crp_cd: 005930,
         crp_cls: Y,
         crp_nm: 삼성전자,
         flr_nm: 삼성전자,
         rcp_dt: 20160531,
         rcp_

응답 결과 필드의 의미는 다음과 같다.

* `err_code` 
 *		에러코드(오류 메시지 참조)
* `err_msg` 
 *		에러메시지(오류 메시지 참조)
* `page_no` 
 *		페이지 번호
* `page_set` 
 *		페이지당 건수
* `total_count` 
 *		총 건수
* `total_page` 
 *		총 페이지 수
* `crp_cls` 
 *	법인구분 : Y(유가), K(코스닥), N(코넥스), E(기타)
* `crp_nm` 
 *	공시대상회사의 종목명(상장사) 또는 법인명(기타법인)
* `crp_cd` 
 *	공시대상회사의 종목코드(6자리) 또는 고유번호(8자리)
* `rpt_nm` 
 *	공시구분+보고서명+기타정보
    * [기재정정] : 본 보고서명으로 이미 제출된 보고서의 기재내용이 변경되어 제출된 것임
    * [첨부정정] : 본 보고서명으로 이미 제출된 보고서의 첨부내용이 변경되어 제출된 것임
    * [첨부추가] : 본 보고서명으로 이미 제출된 보고서의 첨부서류가 추가되어 제출된 것임
    * [변경등록] : 본 보고서명으로 이미 제출된 보고서의 유동화계획이 변경되어 제출된 것임
    * [연장결정] : 본 보고서명으로 이미 제출된 보고서의 신탁계약이 연장되어 제출된 것임
    * [발행조건확정] : 본 보고서명으로 이미 제출된 보고서의 유가증권 발행조건이 확정되어 제출된 것임
    * [정정명령부과] : 본 보고서에 대하여 금융감독원이 정정명령을 부과한 것임
    * [정정제출요구] : 본 보고서에 대하여 금융감독원이 정정제출요구을 부과한 것임
* `rcp_no` 
 *	접수번호(공시뷰어 연결에 이용)
    * PC용 : http://dart.fss.or.kr/dsaf001/main.do?rcpNo=접수번호
    * 모바일용 : http://m.dart.fss.or.kr/html_mdart/MD1007.html?rcpNo=접수번호
* `flr_nm` 
 *	공시 제출인명
* `rcp_dt` 
 *	공시 접수일자(YYYYMMDD)
* `rmk` 
 *	조합된 문자로 각각은 아래와 같은 의미가 있음
    * 유 : 본 공시사항은 한국거래소 유가증권시장본부 소관임
    * 코 : 본 공시사항은 한국거래소 코스닥시장본부 소관임
    * 채 : 본 문서는 한국거래소 채권상장법인 공시사항임
    * 넥 : 본 문서는 한국거래소 코넥스시장 소관임
    * 공 : 본 공시사항은 공정거래위원회 소관임
    * 연 : 본 보고서는 연결부분을 포함한 것임
    * 정 : 본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람
    * 철 : 본 보고서는 철회(간주)되었으니 관련 철회신고서(철회간주안내)를 참고하시기 바람

검색된 정보 항목에서 가장 중요한 것은 접수 번호인 `rcp_no` 이다. 이 값을 알고 있으면 다음 주소를 사용하여 실제 공시 정보를 찾아볼 수 있다.

* http://dart.fss.or.kr/dsaf001/main.do?rcpNo=xxx

예를 들어 삼성전자 분기보고서 접수번호는 20160516003174 이므로 다음 웹사이트를 연결하면 된다.

* http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516003174

## 공시 정보 파싱

공시 정보는 위에서 구한 주소로 접근할 수 있지만 핵심 제무제표를 얻기 위해서는 dcm 번호를 알아야 한다. 현재 dcm 번호는 다음과 같이 찾아낼 수 있다.

In [71]:
import requests
import lxml.html
import re

url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516003174"
req = requests.get(url)
tree = lxml.html.fromstring(req.text)
onclick = tree.xpath('//*[@id="north"]/div[2]/ul/li[1]/a')[0].attrib['onclick']
pattern = re.compile("^openPdfDownload\('\d+',\s*'(\d+)'\)")
dcm = pattern.search(onclick).group(1)

In [72]:
dcm

'5146351'

## 제무 제표 다운로드

정기보고서 등의 제무제표가 포함된 공시에서 dcm 번호를 알고 있다면 다음과 같이 XBRL 정보 혹은 Excel 파일을 다운로드 받을 수 있다.

* http://dart.fss.or.kr/pdf/download/ifrs.do?lang=ko&rcp_no=20160516003174&dcm_no=5146351
* http://dart.fss.or.kr/pdf/download/excel.do?lang=ko&rcp_no=20160516003174&dcm_no=5146351

In [73]:
%cd /home/dockeruser/data/darts

/home/dockeruser/data/darts


In [74]:
%ls

20160516003174_ko.xls


In [97]:
pd.read_excel('/home/dockeruser/data/darts/20160516003174_ko.xls',
             sheetname=u'\uc5f0\uacb0 \ud3ec\uad04\uc190\uc775\uacc4\uc0b0\uc11c',
             skiprows=7, index_col=0, header=[0, 1]).fillna("")

제 48 기 1분기          제 47 기 1분기            \
                                          3개월       누적        3개월        누적   
당기순이익(손실)                             5252803  5252803    4625815   4625815   
기타포괄손익                                -161195  -161195    -819980   -819980   
    후속적으로 당기손익으로 재분류되지 않는 포괄손익         -22420   -22420     -24548    -24548   
        순확정급여부채 재측정요소                  -22783   -22783     -24103    -24103   
        관계기업 및 공동기업의 기타포괄손익에 대한 지분        363      363       -445      -445   
    후속적으로 당기손익으로 재분류되는 포괄손익           -138775  -138775    -795432   -795432   
        매도가능금융자산평가손익                    74595    74595     264851    264851   
        관계기업 및 공동기업의 기타포괄손익에 대한 지분    -111942  -111942     -43607    -43607   
        해외사업장환산외환차이                   -101428  -101428   -1016676  -1016676   
총포괄손익                                 5091608  5091608    3805835   3805835   
포괄손익의 귀속                                                                      
    지배기업 소유주지분                        5070139  5070139    3685109   3685109   
    비지배지분                               21469    21469     120726    120726   

                                      제 47 기    제 46 기  
                                          누적        누적  
당기순이익(손실)                           19060144  23394358  
기타포괄손익                                 76071  -1993414  
    후속적으로 당기손익으로 재분류되지 않는 포괄손익        288047   -720945  
        순확정급여부채 재측정요소                 263978   -710318  
        관계기업 및 공동기업의 기타포괄손익에 대한 지분     24069    -10627  
    후속적으로 당기손익으로 재분류되는 포괄손익          -211976  -1272469  
        매도가능금융자산평가손익                 -414961   -232105  
        관계기업 및 공동기업의 기타포괄손익에 대한 지분    -65330   -118305  
        해외사업장환산외환차이                   268315   -922059  
총포괄손익                               19136215  21400944  
포괄손익의 귀속                                                
    지배기업 소유주지분                      18804189  20990732  
    비지배지분                             332026    410212